In [67]:
import pandas as pd
import re
import pdfplumber as pr
import os
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [11]:
'''
1. 去pdf檔案中找到所以的資訊(表格名稱)，如果已存在程式碼中則不需要改變，但如果不再程式碼中需要新增
2. 新增的步驟是 先找到PDF中明確的表格名稱，將擁有這個表格名稱的內容進行文字取代，取代為你想要的表格名稱，再將這個表格名稱加入LIST內
3. 每個都完成上述1,2步驟後，一路直行到底，然後去final_check.csv這個檔案看他有沒有抓到所以得新資料
4. 去 下半部有old標記無可能以刪除的.csv這個檔案看看最後一欄有old標記的是否真的已刪除
5. 去 page3.csv的檔案將['category','investment Type','Asset Class']欄位為空白的補上，這些代表是新增的
6. ranking為'-'代表沒變，要去上個月的檔案找資料來填
'''

"\n1. 去pdf檔案中找到所以的資訊(表格名稱)，如果已存在程式碼中則不需要改變，但如果不再程式碼中需要新增\n2. 新增的步驟是 先找到PDF中明確的表格名稱，將擁有這個表格名稱的內容進行文字取代，取代為你想要的表格名稱，再將這個表格名稱加入LIST內\n3. 每個都完成上述1,2步驟後，一路直行到底，然後去final_check.csv這個檔案看他有沒有抓到所以得新資料\n4. 去 下半部有old標記無可能以刪除的.csv這個檔案看看最後一欄有old標記的是否真的已刪除\n5. 去 page3.csv的檔案將['category','investment Type','Asset Class']欄位為空白的補上，這些代表是新增的\n6. ranking為'-'代表沒變，要去上個月的檔案找資料來填\n"

In [68]:
PDF_files = []
input_path = r'../input'
for x in os.listdir(input_path):
    if '.pdf' and '年4月' in x:
        PDF_files.append(x)
#這個檔案不要刪除，很重要
GFI = pd.read_excel(r'../input\202204政府基金運用資訊 v2.0.xlsx',sheet_name = 'Onshore (Raw data)',skiprows=1)

In [69]:
#勞退新制
LPF_LIST_ORI = GFI[GFI['Fund']=='LPF']['Mandate Type'].unique()
#勞退舊制
LRF_LIST_ORI = GFI[GFI['Fund']=='LRF']['Mandate Type'].unique()
#勞保
LIF_LIST = GFI[GFI['Fund']=='LIF']['Mandate Type'].unique()
#國保
NPIF_LIST = GFI[GFI['Fund']=='NPIF']['Mandate Type'].unique()
#退撫
PSPF_LIST = GFI[GFI['Fund']=='PSPF']['Mandate Type'].unique()

# 查詢

In [70]:
勞退_PDF = pr.open(input_path+'/勞工退休基金-111年4月.pdf')
#從0開始
#是一個list
ps              = 勞退_PDF.pages
#用range配len跑for迴圈
pg = ps[6]
page_text = pg.extract_text()
print(page_text[500:])
print('-'*50)
print('有無在該頁')
print('97 年第一次委託經營(續約 4)' in page_text)

 -8.59  2  4.56 
合計  25,370,898,094 24,009,830,152  -13.33  -5.36  － 
註：本批次為絕對報酬型，契約目標報酬率係以臺灣證交所公告之最近4年年底股票集中市場平均殖利
率加計200個基本點為基準。 
99 年第二次委託經營(續約 2)             
單位：新臺幣元 
委任迄今 
111年截至4月 
（107.12.3~111.4.30） 
投信 
委託金額  目前淨資產  累積投資報酬
名稱  投資  委託經 目標報酬率
(%) 
報酬率(%) 營排名  (%) 
國泰  7,441,043,825 10,142,401,766  1.76  38.53  －  20.84 
註：本批次為絕對報酬型，契約目標報酬率係以臺灣證交所公告之最近4年年底股票集中市場平均殖利
率加計200個基本點為基準。 
100 年第一次委託經營(續約 2)(相對報酬)     
單位：新臺幣元 
委任迄今 
111年截至4月 
投信  （108.7.15~111.4.30） 
委託金額  目前淨資產  累積投資報酬
名稱  投資  委託經 指標報酬率
(%) 
報酬率(%) 營排名  (%) 
復華  11,295,508,988  25,196,585,328  -7.19  72.87  －  75.42 
  註：本批次為相對報酬型，以臺灣就業99指數為指定指標。 
  7 
--------------------------------------------------
有無在該頁
True


In [71]:
LPF_LIST_ORI

array(['96年第一次委託經營（續約4）', '97年第一次委託經營（續約4）', '98 年第一次委託經營（續約4）',
       '99 年第二次委託經營（續約2） ', '100 年第一次委託經營( 續約2 )', '101 年第一次委託經營( 續約 2)',
       '101 年第二次委託經營( 續約 2)', '102 年第二次委託經營 ( 續約 2)',
       '103 年第一次委託經營第二期(續約)', '103 年第二次委託經營', '104 年第一次委託經營第一期( 續約 )',
       '106 年第一次委託經營', '107 年第一次委託經營', '107 年第二次委託經營', '109 年第一次委託經營',
       '110年第一次委託經營'], dtype=object)

In [72]:
LPF_LIST = []
LRF_LIST = []
有新制勞退目標基金的頁數 = []
新制勞退目標基金的名稱   = []
NEW_FUND_NAMES         = []

有舊制勞退目標基金的頁數 = []
舊制勞退目標基金的名稱   = []
OLD_FUND_NAMES         = []

for PDF_file in PDF_files:
    if '勞工退休基金' in PDF_file and '.pdf' in PDF_file:
        勞退_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞退_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            if p < 11:
                #新制，將PDF檔案中的基金名稱改成excel內的
                FUND_NAME = 'LPF'
                if '96 年第一次委託經營(續約 4)' in page_text:
                    print('96 年第一次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('96 年第一次委託經營(續約 4)','96年第一次委託經營（續約4）')
                    LPF_LIST.append('96年第一次委託經營（續約4）')
                if '97 年第一次委託經營(續約 4)' in page_text:
                    print('97 年第一次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97 年第一次委託經營(續約 4)','97年第一次委託經營（續約4）')
                    LPF_LIST.append('97年第一次委託經營（續約4）')
                if '98 年第一次委託經營(續約 4)' in page_text:
                    print('98 年第一次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('98 年第一次委託經營(續約 4)','98 年第一次委託經營（續約4）')
                    LPF_LIST.append('98 年第一次委託經營（續約4）')
                if '99 年第二次委託經營(續約 2)' in page_text:
                    print('99 年第二次委託經營(續約 2) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('99 年第二次委託經營(續約 2)','99 年第二次委託經營（續約2） ')
                    LPF_LIST.append('99 年第二次委託經營（續約2） ')
                if '100 年第一次委託經營(續約 2)(相對報酬)' in page_text:
                    print('100 年第一次委託經營(續約 2)(相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100 年第一次委託經營(續約 2)(相對報酬)','100 年第一次委託經營( 續約2 )')
                    LPF_LIST.append('98-1 全球被動股票型（ 續約2）')
                if '101 年第一次委託經營(續約 2)(相對報酬)' in page_text:
                    print('101 年第一次委託經營(續約 2)(相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101 年第一次委託經營(續約 2)(相對報酬)','101 年第一次委託經營( 續約 2)')
                    LPF_LIST.append('101 年第一次委託經營( 續約 2)')
                if '101 年第二次委託經營(續約 2)' in page_text:
                    print('101 年第二次委託經營(續約 2) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101 年第二次委託經營(續約 2)','101 年第二次委託經營( 續約 2)')
                    LPF_LIST.append('101 年第二次委託經營( 續約 2)')
                if '100-1 全球不動產有價證券型（續約）' in page_text:
                    print('100-1 全球不動產有價證券型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球不動產有價證券型（續約）','100-1 全球不動產有價證券型（續約）')
                    LPF_LIST.append('100-1 全球不動產有價證券型（續約）')
                if '102 年第二次委託經營(續約 2) (相對報酬)' in page_text:
                    print('102 年第二次委託經營(續約 2) (相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102 年第二次委託經營(續約 2) (相對報酬)','102 年第二次委託經營 ( 續約 2)')
                    LPF_LIST.append('102 年第二次委託經營 ( 續約 2)')
                if '103 年第一次委託經營第二期(續約)' in page_text:
                    print('103 年第一次委託經營第二期(續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('103 年第一次委託經營第二期(續約)','103 年第一次委託經營第二期(續約)')
                    LPF_LIST.append('103 年第一次委託經營第二期(續約)')
                if '103 年第二次委託經營(續約) (相對報酬)' in page_text:
                    print('103 年第二次委託經營(續約) (相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('103 年第二次委託經營(續約) (相對報酬)','103 年第二次委託經營')
                    LPF_LIST.append('103 年第二次委託經營')
                if '104 年第一次委託經營  (續約)' in page_text:
                    print('104 年第一次委託經營  (續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('104 年第一次委託經營  (續約)','104 年第一次委託經營第一期( 續約 )')
                    LPF_LIST.append('104 年第一次委託經營第一期( 續約 )')
                if '106 年第一次委託經營' in page_text:
                    print('106 年第一次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('106 年第一次委託經營','106 年第一次委託經營')
                    LPF_LIST.append('106 年第一次委託經營')
                if '107 年第一次委託經營(相對報酬)' in page_text:
                    print('107 年第一次委託經營(相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('107 年第一次委託經營(相對報酬)','107 年第一次委託經營')
                    LPF_LIST.append('107 年第一次委託經營')            
                if '107 年第二次委託經營' in page_text:
                    print('107 年第二次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('107 年第二次委託經營','107 年第二次委託經營')
                    LPF_LIST.append('107 年第二次委託經營')  
                if '109 年第一次委託經營' in page_text:
                    print('109 年第一次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('109 年第一次委託經營','109 年第一次委託經營')
                    LPF_LIST.append('109 年第一次委託經營')  
                 


            elif p == 11:
                print('到達12頁')
                if '110 年第一次委託經營' in page_text:
                    print('110 年第一次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('110 年第一次委託經營','110年第一次委託經營')
                    LPF_LIST.append('110年第一次委託經營')
                #舊制，將PDF檔案中的基金名稱改成excel內的，過渡期
                FUND_NAME = 'LRF'
                if '國內 98 年第二次委託經營(續約 4)' in page_text:
                    print('國內 98 年第二次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 98 年第二次委託經營(續約 4)','國內 98 年第二次委託經營( 續約 4)')
                    LRF_LIST.append('國內 98 年第二次委託經營( 續約 4)')
                if '97-1 全球增值債券型(續約 3)' in page_text:
                    print('97-1 全球增值債券型(續約 3)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-1 全球增值債券型(續約 3)','97年續約3國外委託經營( 全球增值債券型 )')
                    LRF_LIST.append('97年續約3國外委託經營( 全球增值債券型 )')      
            elif p>11:
                FUND_NAME = 'LRF'      
                if '國內 101 年第一次委託經營(相對報酬)(續約 2)' in page_text:
                    print('國內 101 年第一次委託經營(相對報酬)(續約 2)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 101 年第一次委託經營(相對報酬)(續約 2)','國內 101 年第一次委託經營( 續約2 )')
                    LRF_LIST.append('國內 101 年第一次委託經營( 續約2 )') 
                if '國內 101 年第二次委託經營(續約 2)' in page_text:
                    print('國內 101 年第二次委託經營(續約 2)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 101 年第二次委託經營(續約 2)','國內 101 年第二次委託經營( 續約2 )')
                    LRF_LIST.append('國內 101 年第二次委託經營( 續約2 )')
                if '國內 104 年第一次委託經營(續約)' in page_text:
                    print('國內 104 年第一次委託經營(續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 104 年第一次委託經營(續約)','國內104 年第一次委託經營第一期( 續約 )')
                    LRF_LIST.append('國內104 年第一次委託經營第一期( 續約 )')
                if '國內 106 年第一次委託經營' in page_text:
                    print('國內 106 年第一次委託經營  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 106 年第一次委託經營','國內 106 年第一次委託經營第一期')
                    LRF_LIST.append('國內 106 年第一次委託經營第一期')
                
            
            for target_fund in list(LPF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        if p<=11:
                            #小作弊，因為第五頁有三張表，第三張才是我要的
                            if p == 5:
                                table_num = 2
                            有新制勞退目標基金的頁數.append((p,table_num))
                            新制勞退目標基金的名稱.append(target_fund)
                            NEW_FUND_NAMES.append(FUND_NAME)
                            table_num+=1
            for target_fund in list(LRF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        if p>=11:
                            #小作弊，因為第五頁有三張表，第三張才是我要的
                            if p == 5:
                                table_num = 2
                            有舊制勞退目標基金的頁數.append((p,table_num))
                            舊制勞退目標基金的名稱.append(target_fund)
                            OLD_FUND_NAMES.append(FUND_NAME)
                            table_num+=1

        LPF = pd.DataFrame({})
        table_name_index = 0
        ps = 勞退_PDF.pages
        for page,table_num in 有新制勞退目標基金的頁數:
            pg = ps[page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_勞退 = pd.DataFrame(page_tables[table_num])
                #處理表格內的資料
                table_勞退.iloc[0] = table_勞退.iloc[0].fillna('',axis=0)
                table_勞退.iloc[1] = table_勞退.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(2)
                table_勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞退.iloc[0,4])
                table_勞退_final = table_勞退.iloc[2:,:]
                table_勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_勞退_final['Target'] = table_勞退_final['Target'].fillna(method = 'ffill')
                table_勞退_final[['Fund','Mandate Type','From','As of']]=['LPF',新制勞退目標基金的名稱[table_name_index],Date_Start,Date_End]
                table_勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_勞退_final = table_勞退_final[table_勞退_final['SITE']!='合計']
                table_name_index += 1
                LPF = LPF.append(table_勞退_final)
            except:
                print(str(page)+'有問題')
                table_name_index += 0
        LPF.reset_index(drop=True,inplace=True)

        LRF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_file)
        table_name_index = 0
        ps = PDF_file.pages
        for page,table_num in 有舊制勞退目標基金的頁數:
            pg = ps[page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_勞退 = pd.DataFrame(page_tables[table_num])
                #處理表格內的資料
                table_勞退.iloc[0] = table_勞退.iloc[0].fillna('',axis=0)
                table_勞退.iloc[1] = table_勞退.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(2)
                table_勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞退.iloc[0,4])
                table_勞退_final = table_勞退.iloc[2:,:]
                table_勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_勞退_final['Target'] = table_勞退_final['Target'].fillna(method = 'ffill')
                table_勞退_final[['Fund','Mandate Type','From','As of']]=['LRF',舊制勞退目標基金的名稱[table_name_index],Date_Start,Date_End]
                table_勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_勞退_final = table_勞退_final[table_勞退_final['SITE']!='合計']
                table_name_index += 1
                LPF = LPF.append(table_勞退_final)
            except:
                print(str(page)+'有問題')
                table_name_index += 0
        LRF.reset_index(drop=True,inplace=True)

96 年第一次委託經營(續約 4) in 6頁
97 年第一次委託經營(續約 4) in 7頁
98 年第一次委託經營(續約 4) in 7頁
99 年第二次委託經營(續約 2) in 7頁
100 年第一次委託經營(續約 2)(相對報酬) in 7頁
101 年第一次委託經營(續約 2)(相對報酬) in 8頁
101 年第二次委託經營(續約 2) in 8頁
102 年第二次委託經營(續約 2) (相對報酬) in 8頁
103 年第一次委託經營第二期(續約) in 9頁
103 年第二次委託經營(續約) (相對報酬) in 9頁
104 年第一次委託經營  (續約) in 9頁
106 年第一次委託經營 in 10頁
107 年第一次委託經營(相對報酬) in 10頁
107 年第二次委託經營 in 11頁
109 年第一次委託經營 in 11頁
到達12頁
110 年第一次委託經營 in 12頁
國內 98 年第二次委託經營(續約 4) in 12頁
國內 101 年第二次委託經營(續約 2)  in 13頁
國內 104 年第一次委託經營(續約)  in 13頁
國內 106 年第一次委託經營  in 13頁


In [73]:
有新制勞退目標基金的頁數

[(5, 2),
 (6, 0),
 (6, 1),
 (6, 2),
 (7, 0),
 (7, 1),
 (7, 2),
 (8, 0),
 (8, 1),
 (8, 2),
 (9, 0),
 (9, 1),
 (10, 0),
 (10, 1),
 (11, 0)]

# 查詢

In [74]:
國保_PDF = pr.open(input_path+'/國民年金保險基金-111年4月.pdf')
#從0開始
#是一個list
ps              = 國保_PDF.pages
#用range配len跑for迴圈
pg = ps[6]
page_text = pg.extract_text()
print(page_text[500:])
print('-'*50)
print('有無在該頁')
print('97 年第一次委託經營(續約 4)' in page_text)

為基準。 
109-1 全球美元公司增值債券型 
                     
單位：美元 
委任迄今 
111 年截至04月 
(110.01.13~111.04.30) 
受託機構  委託金額  目前淨資產  累積投資報酬率
投資報酬率 指標報酬率
(％)  排名 
(％)  (％) 
DWS  60,000,000  53,701,432  -11.88  -12.09  2 
INSIGHT  60,000,000  53,718,293  -11.90  -11.92  1 
JP Morgan  60,000,000  53,654,265  -12.01  -12.23  3  -11.68 
PIMCO  60,000,000  53,411,860  -12.35  -12.56  4 
WESTERN  60,000,000  53,248,685  -12.55  -12.64  5 
合計  300,000,000  267,734,535  -12.14  -12.29  －  - 
   
7 
 
--------------------------------------------------
有無在該頁
False


In [75]:
NPIF_LIST = []
有國保目標基金的頁數 = []
國保目標基金的名稱   = []
NPIF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '國民年金保險' in PDF_file and '.pdf' in PDF_file:
        國保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 國保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'NPIF'
            if '102 年度續約 1 委託帳戶' in page_text:
                print('102 年度續約 1 委託帳戶 in ' + str(p+1) + '頁')
                page_text = page_text.replace('102 年度續約 1 委託帳戶','國民年金保險基金102 年度委託帳戶續約1')
                NPIF_LIST.append('國民年金保險基金102 年度委託帳戶續約1')


            for target_fund in list(NPIF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有國保目標基金的頁數.append((p,table_num))
                        國保目標基金的名稱.append(target_fund)
                        NPIF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

    NPIF = pd.DataFrame({})
    PDF_file = pr.open(input_path+'/'+'國民年金保險基金-111年4月.pdf')
    table_name_index = 0
    ps = PDF_file.pages
    for page, table_num in 有國保目標基金的頁數:
        pg = ps[page]
        page_tables = pg.extract_tables()
        try:
            table_國民年金 = pd.DataFrame(page_tables[table_num])
            #處理表格內的資料
            table_國民年金.iloc[0] = table_國民年金.iloc[0].fillna('',axis=0)
            table_國民年金.iloc[1] = table_國民年金.iloc[1].fillna('',axis=0)
            Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_國民年金.iloc[0,4]).group(1)
            Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_國民年金.iloc[0,4]).group(2)
            table_國民年金.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_國民年金.iloc[0,4])
            table_國民年金_final = table_國民年金.iloc[2:,:]
            table_國民年金_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
            table_國民年金_final['Target'] = table_國民年金_final['Target'].fillna(method = 'ffill')
            table_國民年金_final[['Fund','Mandate Type','From','As of']]=['NPIF',國保目標基金的名稱[table_name_index],Date_Start,Date_End]
            table_國民年金_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
            table_國民年金_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
            table_國民年金_final = table_國民年金_final[table_國民年金_final['SITE']!='合計']
            table_name_index += 1
        except:
            table_name_index += 0
        NPIF = NPIF.append(table_國民年金_final)

102 年度續約 1 委託帳戶 in 4頁


# 查詢

In [76]:
for PDF_file in PDF_files:
    if '基金運用情形' in PDF_file and '.pdf' in PDF_file:
        退撫_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 退撫_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[3]
            page_text = pg.extract_text()
            print(page_text)

 
 
  107年委託經營（自 109年3月24日至 111年4月30日止） 
111年度 委託迄今 
受託機構名稱  委託金額  資產淨值  報酬率 報酬率 委託經 指標報酬率
(%)  (%)  營排名  (%) 
群益投信  2,000,000,000    3,162,261,657  -8.64   81.58  1  
國泰投信  2,000,000,000    3,118,661,785  -10.11    80.89  2  
復華投信  2,000,000,000    3,151,106,978  -11.15    79.87  3  
統一投信  2,000,000,000    3,149,801,237  -11.24    79.12  4   98.57 
野村投信  2,000,000,000    3,055,856,619  -12.25   77.61  5  
保德信投信  2,000,000,000    3,077,716,491  -10.05   75.03  6  
合計  12,000,000,000  18,715,404,767  -10.58  79.04    
註：本批次委託經營係追踨指數(相對報酬型)，以台灣證券交易所編製之發行量加權股價報酬指數之 
投資報酬率加50個基本點為年投資報酬目標。 
 
 
108年委託經營（自 109年2月7日至 111年4月30日止） 
111年度 委託迄今 
受託機構名稱  委託金額  資產淨值  報酬率 報酬率 委託經 目標報酬率
(%)  (%)  營排名  (%) 
安聯投信  5,000,000,000  6,628,291,445  -10.19   44.08  1  
國泰投信  5,000,000,000  6,411,901,438  -13.71   37.38  2  
野村投信  5,000,000,000  6,376,473,720  -14.64   36.59  3   13.55 
復華投信  4,000,000,000  5,327,590,362  -19.61   34.45  4  
合計  19,000,000,000  24,744,256,965  -14.40   38.15    
註：本批次委託經營係非追踨

In [77]:
PSPF_LIST = []
有退撫目標基金的頁數 = []
退撫目標基金的名稱   = []
PSPF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '基金運用情形' in PDF_file and '.pdf' in PDF_file:
        退撫_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 退撫_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'PSPF'
            if '101年委託經營' in page_text:
                print('101年委託經營 in ' + str(p+1) + '頁')
                page_text = page_text.replace('101年委託經營','101年委託經營')
                PSPF_LIST.append('101年委託經營')
            if '102年委託經營' in page_text:
                print('102年委託經營 in ' + str(p+1) + '頁')
                page_text = page_text.replace('102年委託經營','102年委託經營')
                PSPF_LIST.append('102年委託經營')
            if '105年委託經營' in page_text:
                print('105年委託經營  in ' + str(p+1) + '頁')
                page_text = page_text.replace('105年委託經營','105年委託經營') 
                PSPF_LIST.append('105年委託經營')
            if '107年委託經營' in page_text:
                print('107年委託經營  in ' + str(p+1) + '頁')
                page_text = page_text.replace('107年委託經營','107年委託經營')    
                PSPF_LIST.append('107年委託經營')            
            if '108年委託經營' in page_text:
                print('108年委託經營 in ' + str(p+1) + '頁')
                page_text = page_text.replace('108年委託經營','108年委託經營')  
                PSPF_LIST.append('108年委託經營')       
            
            for target_fund in list(PSPF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有退撫目標基金的頁數.append((p,table_num))
                        退撫目標基金的名稱.append(target_fund)
                        PSPF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

    PSPF = pd.DataFrame({})
    PSPF_table_name_index = 0
    for PSPF_page,PSPF_table_num in 有退撫目標基金的頁數:
        pg = ps[PSPF_page]
        page_tables = pg.extract_tables()
        #print(len(page_tables))
        try:
            table_委託經營  = pd.DataFrame(page_tables[PSPF_table_num])
            multi          = [table_委託經營.iloc[0,:].dropna(axis=0),table_委託經營.iloc[1,:].fillna('')+table_委託經營.iloc[2,:].fillna('')]
            mindex         = pd.MultiIndex.from_product(multi,names=['Level1','Level2'])
            table_最終結果  = pd.DataFrame(columns = mindex, index = table_委託經營.iloc[3:,:].index)
            table_最終結果[table_委託經營.iloc[0,:].dropna(axis=0)[0]] = table_委託經營.iloc[3:,:].values

            #將資料補齊
            #找到基金的日期
            Funds_Date     = table_最終結果.columns.get_level_values(0)[0]
            table_最終結果  = table_最終結果[table_最終結果[Funds_Date,'受託機構名稱'] != '合計']
            table_最終結果[Funds_Date,'Fund'] = 'PSPF'
            table_最終結果[Funds_Date,'Mandate Type'] = Funds_Date.split('（')[0]
            #將報酬率補起來，有的是目標，有的是指標
            try:
                table_最終結果[Funds_Date,'目標報酬率\n(%)'] = table_最終結果[Funds_Date,'目標報酬率\n(%)'].fillna(method = 'ffill')
            except:
                table_最終結果[Funds_Date,'指標報酬率\n(%)'] = table_最終結果[Funds_Date,'指標報酬率\n(%)'].fillna(method = 'ffill')
            table_最終結果[Funds_Date,'受託機構名稱'] = table_最終結果[Funds_Date,'受託機構名稱'].str.replace('投信','')
            #補開始及結束的日期
            table_最終結果[Funds_Date,'From']  = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][3:])).replace('日','')
            table_最終結果[Funds_Date,'As of'] = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][3:])).replace('日','')
            for target in ['年','月']:
                table_最終結果[Funds_Date,'From']  = table_最終結果[Funds_Date,'From'].str.replace(target,'/')
                table_最終結果[Funds_Date,'As of'] = table_最終結果[Funds_Date,'As of'].str.replace(target,'/')
            PSPF = PSPF.append(pd.DataFrame(table_最終結果.values))
        except:
            pass
    PSPF.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','Fund','Mandate Type','From','As of']

101年委託經營 in 3頁
102年委託經營 in 3頁
105年委託經營  in 3頁
107年委託經營  in 4頁
108年委託經營 in 4頁


# 查詢

In [78]:
for PDF_file in PDF_files:
    if '勞工保險' in PDF_file and '.pdf' in PDF_file:
        勞保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[3]
            page_text = pg.extract_text()
print(page_text[-600:])

殖利率加計
200個基本點為基準。 
 
                  勞工保險基金 106 年度第 1 次       
單位：新臺幣元 
委任迄今 
111年截至4月
(107.10.8~111.4.30) 
投信名稱  委託金額  目前淨資產  累積投資報酬率
投資  目標報酬率
(％)  排名 
報酬率(％)  (％) 
台新  1,000,000,000  1,493,480,679  -11.49  49.35  2 
安聯  1,000,000,000  1,662,723,730  -9.79  66.27  1 
保德信  1,000,000,000  1,300,379,592  -8.78  30.04  6 
國泰  900,000,000  1,272,914,519  1.62  42.99  4  21.76 
群益  1,000,000,000  1,375,073,126  -9.07  37.51  5 
滙豐中華  1,000,000,000  1,430,573,302  -2.33  43.06  3 
合計  5,900,000,000  8,535,144,948  -7.08  44.73  - 
註：本批次為絕對報酬型，契約目標報酬率係以臺灣證交所公告之最近5年年底股票集中市場平均殖利率加計
200個基本點為基準。 
4 
 


In [79]:
LIF_LIST = []
有勞保目標基金的頁數 = []
勞保目標基金的名稱   = []
LIF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '勞工保險' in PDF_file and '.pdf' in PDF_file:
        勞保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'LIF'
            if '勞工保險基金 103 年度續約 3' in page_text:
                print('勞工保險基金 103 年度續約 3 in ' + str(p+1) + '頁')
                page_text = page_text.replace('勞工保險基金 103 年度續約 3','勞工保險基金 103年度續約3')
                LIF_LIST.append('勞工保險基金 103年度續約3')
            if '勞工保險基金 104 年度續約 3' in page_text:
                print('勞工保險基金 104 年度續約 3 in ' + str(p+1) + '頁')
                page_text = page_text.replace('勞工保險基金 104 年度續約 3','勞工保險基金 104年度續約3')
                LIF_LIST.append('勞工保險基金 104年度續約3')
            if '勞工保險基金 106 年度第 1 次' in page_text:
                print('勞工保險基金 106 年度第 1 次  in ' + str(p+1) + '頁')
                page_text = page_text.replace('勞工保險基金 106 年度第 1 次','勞工保險基金 106 年度年第一次委託經營') 
                LIF_LIST.append('勞工保險基金 106 年度年第一次委託經營')       
            
                
            page_text = page_text.replace('\n','')
            for target_fund in list(LIF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有勞保目標基金的頁數.append((p,table_num))
                        勞保目標基金的名稱.append(target_fund)
                        LIF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

    LIF = pd.DataFrame({})
    PDF_file = pr.open(input_path+'/'+PDF_files[2])
    table_num_index=0
    for page_table, table_num in 有勞保目標基金的頁數:
        ps = PDF_file.pages
        pg = ps[page_table]
        table = pg.extract_tables()[table_num]
        table_勞保 = pd.DataFrame(table)
        #處理表格內的資料
        table_勞保.iloc[0] = table_勞保.iloc[0].fillna('',axis=0)
        table_勞保.iloc[1] = table_勞保.iloc[1].fillna('',axis=0)
        Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞保.iloc[0,4]).group(1)
        Date_End   = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞保.iloc[0,4]).group(2)
        table_勞保.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞保.iloc[0,4])
        table_勞保_final           = table_勞保.iloc[2:,:]
        table_勞保_final.columns   = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
        table_勞保_final['Target'] = table_勞保_final['Target'].fillna(method = 'ffill')
        table_勞保_final[['Fund','Mandate Type','From','As of']]=['LIF',勞保目標基金的名稱[table_num_index],Date_Start,Date_End]
        table_勞保_final['From']   = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
        table_勞保_final['As of']  = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
        table_勞保_final = table_勞保_final[table_勞保_final['SITE']!='合計']
        table_num_index+=1
        LIF = LIF.append(table_勞保_final)

勞工保險基金 103 年度續約 3 in 4頁
勞工保險基金 104 年度續約 3 in 4頁
勞工保險基金 106 年度第 1 次  in 4頁


# 要把新舊合併

In [125]:
#他是看新月份的東西有沒有全抓到
GFI['新舊'] = 'old'
v=Final_df = pd.concat([LPF,LRF,LIF,NPIF,PSPF],axis=0)
Final_df.reset_index(drop=True,inplace=True)
Final_df['SITE'] = Final_df['SITE'].str.replace('\n','')
Final_df['SITE'] = Final_df['SITE'].str.replace('滙豐中華','匯豐中華')
Final_df = Final_df[['Fund','Mandate Type','SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','From','As of']]
Final_df.to_excel(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2_final_check.xlsx',encoding='UTF-8-SIG',index=False,header=True)

In [126]:
新舊合併 = pd.concat([Final_df,GFI],axis=0)
新舊合併 = 新舊合併[pd.isna(新舊合併['Fund'])==False]
for col in ['Fund','Mandate Type','SITE']:
    新舊合併[col] = 新舊合併[col].str.strip()
新舊合併_去重複 = 新舊合併.drop_duplicates(subset = ['Fund','Mandate Type','SITE'],keep = 'first')
新舊合併_去重複.reset_index(drop=True,inplace=True)

In [127]:
新舊合併_去重複_new = 新舊合併_去重複[新舊合併_去重複['新舊'] != 'old']

In [128]:
GFI

,Fund,Mandate Type,Asset Class,SITE,Mandate Size,AUM,YTD %,Since Inception %,Ranking,Target,From,As of,benchmark,新舊
0,LPF,96年第一次委託經營（續約4）,Absolute Return,野村,"23,580,874,211","32,235,976,725",-14.66,42.07,1.0,15.63,2019-09-27,2022-04-30,NaN,old
1,LPF,97年第一次委託經營（續約4）,Absolute Return,野村,"9,193,179,186","11,885,257,401",-14.52,29.28,1.0,13.62,2020-01-20,2022-04-30,NaN,old
2,LPF,98 年第一次委託經營（續約4）,Absolute Return,野村,"19,093,323,980","18,271,737,023",-14.76,-4.30,1,4.56,2021-07-16,2022-04-30,NaN,old
3,LPF,98 年第一次委託經營（續約4）,Absolute Return,保德信,"6,277,574,114","5,738,093,129",-8.47,-8.59,2,4.56,2021-07-16,2022-04-30,NaN,old
4,LPF,99 年第二次委託經營（續約2）,Absolute Return,國泰,"7,441,043,825","10,142,401,766",1.76,38.53,1.0,20.84,2018-12-03,2022-04-30,NaN,old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,PSPF,107年委託經營,Relative Return,保德信,"2,000,000,000","3,077,716,491",-10.05,75.03,6,98.57,2020-03-24,2022-04-30,NaN,old
89,PSPF,108年委託經營,Absolute Return,安聯,"5,000,000,000","6,628,291,445",-10.19,44.08,1,13.55,2020-02-07,2022-04-30,NaN,old
90,PSPF,108年委託經營,Absolute Return,國泰,"5,000,000,000","6,411,901,438",-13.71,37.38,2,13.55,2020-02-07,2022-04-30,NaN,old
91,PSPF,108年委託經營,Absolute Return,野村,"5,000,000,000","6,376,473,720",-14.64,36.59,3,13.55,2020-02-07,2022-04-30,NaN,old


In [129]:
#先skip
#20220604修正更新資料
GFI['SITE'] = GFI['SITE'].astype('str')
GFI         = GFI[~pd.isna(GFI['Mandate Type'])]
新舊合併_去重複_new['SITE'] = 新舊合併_去重複_new['SITE'].astype('str')
match  = []
a      = []
for x in range(len(GFI)):
    for y in range(len(新舊合併_去重複_new)):
        if GFI.loc[x,'Fund'].strip().lower() == 新舊合併_去重複_new.loc[y,'Fund'].strip().lower() and GFI.loc[x,'Mandate Type'].strip().lower() == 新舊合併_去重複_new.loc[y,'Mandate Type'].strip().lower() and GFI.loc[x,'SITE'].strip().lower() == 新舊合併_去重複_new.loc[y,'SITE'].strip().lower():
            #排名不更新
            match.append((x,y))
            a.append(y)
            if 新舊合併_去重複_new.loc[y,'Ranking'] == '－' or 新舊合併_去重複_new.loc[y,'Ranking'] == '–' or 新舊合併_去重複_new.loc[y,'Ranking'] == '-' or Final_df.loc[y,'Ranking'] == '－':
                #print('ranking no change')
                要更新的資料 = list(新舊合併_去重複_new.loc[y,['Mandate Size','AUM','YTD %','Since Inception %','Target','As of']].values)
                GFI.loc[x,['Mandate Size','AUM','YTD %','Since Inception %','Target','As of']] = 要更新的資料
            else:
                #print('ranking change')
                要更新的資料 = list(新舊合併_去重複_new.loc[y,['Mandate Size','AUM','YTD %','Since Inception %','Target','As of','Ranking']].values)
                GFI.loc[x,['Mandate Size','AUM','YTD %','Since Inception %','Target','As of','Ranking']] = 要更新的資料
            break
        else:
            pass

In [132]:
#我在上一個cell先挑出match到舊資料的本月份資料的INDEX
#因此如果 b for b in a if b not in 新舊合併_去重複_new.index 代表是MATCH不到的資料，但如果我用新舊合併_去重複會將OLD的資料也放進這個LIST
#對後續更新不利
新增資料的row = [b for b in a if b not in 新舊合併_去重複_new.index]

In [121]:
新舊合併_去重複.loc[新增資料的row,'新舊'] = 'new'
新舊合併_去重複.to_excel(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2_下半部有old標記為可能已刪除的.xlsx',encoding='UTF-8-SIG',index=False,header=True)

,Fund,Mandate Type,SITE,Mandate Size,AUM,YTD %,Since Inception %,Ranking,Target,From,As of,Asset Class,benchmark,新舊
0,LPF,96年第一次委託經營（續約4）,野村,"23,580,874,211","32,235,976,725",-14.66,42.07,－,15.63,2019/9/27,2022/4/30,NaN,NaN,NaN
1,LPF,97年第一次委託經營（續約4）,野村,"9,193,179,186","11,885,257,401",-14.52,29.28,－,13.62,2020/1/20,2022/4/30,NaN,NaN,NaN
2,LPF,98 年第一次委託經營（續約4）,野村,"19,093,323,980","18,271,737,023",-14.76,-4.30,1,4.56,2021/7/16,2022/4/30,NaN,NaN,NaN
3,LPF,98 年第一次委託經營（續約4）,保德信,"6,277,574,114","5,738,093,129",-8.47,-8.59,2,4.56,2021/7/16,2022/4/30,NaN,NaN,NaN
4,LPF,99 年第二次委託經營（續約2）,國泰,"7,441,043,825","10,142,401,766",1.76,38.53,－,20.84,2018/12/3,2022/4/30,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,PSPF,108年委託經營,國泰,"5,000,000,000","6,411,901,438",-13.71,37.38,2,13.55,2020/2/7,2022/4/30,NaN,NaN,NaN
89,PSPF,108年委託經營,野村,"5,000,000,000","6,376,473,720",-14.64,36.59,3,13.55,2020/2/7,2022/4/30,NaN,NaN,NaN
90,PSPF,108年委託經營,復華,"4,000,000,000","5,327,590,362",-19.61,34.45,4,13.55,2020/2/7,2022/4/30,NaN,NaN,NaN
91,LPF,100 年第一次委託經營( 續約2 ),復華,11295508988,26857175824,-1.08,84.27,1.0,83.74,2019-07-15 00:00:00,2022-03-31 00:00:00,Relative Return,NaN,old


In [63]:
GFI.to_excel(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2_Final.xlsx',encoding='UTF-8-SIG',index=False,header=True)